In [1]:
import logging
import math,random
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from models.Attention import Attention
from models.Embeddings.BERT import BERT_Embedding
from models.Embeddings.GLOVE import GLOVE_Embedding
from models.Encoders.CNN import CNN_Encoder
from models.Encoders.RNN import RNN_User_Encoder
from models.Interactors.CNN import CNN_Interactor
from models.Interactors.FIM import FIM_Interactor
from models.Interactors.KNRM import KNRM_Interactor
from models.Interactors.BERT_Overlook import BERT_Interactor
from models.Interactors.BERT_Onepass import BERT_Interactor

from models.ESM import ESM

from data.configs.demo import config
from utils.utils import prepare
from utils.Manager import Manager

from models.Modules.DRM import DRM_Matching
# from models.Modules.TFM import TFM

In [2]:
config.his_size = 10
config.embedding = 'bert'
config.hidden_dim = 768
config.device = 1
config.learning_rate = 1e-6

# config.path = "C:/"

manager = Manager(config)

vocab, loaders = prepare(manager)
record = next(iter(loaders[0]))

[2021-08-13 19:46:30,728] INFO (root) Hyper Parameters are scale:demo
mode:tune
epochs:8
batch_size:10
k:3
threshold:0
title_length:20
abs_length:40
signal_length:50
npratio:4
his_size:10
dropout_p:0.2
device:1
lr:0.0001
bert_lr:3e-05
metrics:auc,mean_mrr,ndcg@5,ndcg@10
embedding:bert
embedding_dim:300
hidden_dim:768
rank:0
world_size:0
step:[0]
seeds:42
interval:10
val_freq:2
schedule:None
path:../../../Data/
tb:False
bert:bert-base-uncased
learning_rate:1e-06
cdd_size:5
[2021-08-13 19:46:30,729] INFO (root) preparing dataset...
Traceback (most recent call last):
  File "/data/workspace/Peitian/nn/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/data/workspace/Peitian/nn/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/data/workspace/Peitian/nn/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/data/workspace/Pei

In [3]:
embedding = BERT_Embedding(manager)
encoderN = CNN_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
docReducer = DRM_Matching(manager)
# termFuser = TFM(manager.his_size, manager.k)
# interactor = CNN_Interactor(manager)

interactor = BERT_Interactor(manager)

esm = ESM(manager, embedding, encoderN, encoderU, docReducer, None, interactor).to(manager.device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationsh

In [4]:
record = next(iter(loaders[1]))

In [7]:
esm.eval()

ESM(
  (embedding): BERT_Embedding(
    (embedding): Embedding(30522, 768, padding_idx=0)
    (layerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropOut): Dropout(p=0.1, inplace=False)
  )
  (encoderN): CNN_Encoder(
    (wordQueryProject): Linear(in_features=768, out_features=768, bias=True)
    (CNN): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=(1,))
    (layerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (RELU): ReLU()
    (Tanh): Tanh()
  )
  (encoderU): RNN_User_Encoder(
    (lstm): LSTM(768, 768, batch_first=True)
  )
  (docReducer): DRM_Matching()
  (interactor): BERT_Interactor(
    (bert): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_f

In [8]:
esm(record)

tensor([[0.4865, 0.4875, 0.4870, 0.4876, 0.4872, 0.4865, 0.4882, 0.4879, 0.4877,
         0.4877, 0.4881, 0.4896, 0.4895, 0.4872, 0.4867, 0.4875, 0.4871, 0.4883,
         0.4868, 0.4883, 0.4875, 0.4866]], device='cuda:1',
       grad_fn=<SigmoidBackward>)